In [1]:
import os

In [2]:
%pwd

'd:\\Visa-Approval-Prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Visa-Approval-Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.VisaApprovalPredict.constants import *
from src.VisaApprovalPredict.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from src.VisaApprovalPredict import logger
from src.VisaApprovalPredict.utils.common import get_size

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Extracted files to {unzip_path}")
        except zipfile.BadZipFile:
            logger.error(f"File {self.config.local_data_file} is not a zip file")
        except Exception as e:
            logger.error(f"An error occurred while extracting the zip file: {e}")

    def process_extracted_files(self):
        # Add your file processing logic here
        pass


In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()

    # Check if the downloaded file is a valid zip file
    if zipfile.is_zipfile(data_ingestion.config.local_data_file):
        data_ingestion.extract_zip_file()
        data_ingestion.process_extracted_files()
    else:
        logger.error("Downloaded file is not a valid zip file.")
except Exception as e:
    logger.error(f"An error occurred: {e}")
    raise e

[2025-01-03 17:19:23,595: INFO: common: yaml file: config\config.yaml retrieved from cache]
[2025-01-03 17:19:23,595: INFO: common: yaml file: params.yaml retrieved from cache]
[2025-01-03 17:19:23,595: INFO: common: yaml file: schema.yaml retrieved from cache]
[2025-01-03 17:19:23,595: INFO: common: Directory already exists at: artifacts]
[2025-01-03 17:19:23,643: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-01-03 17:19:25,591: INFO: 635984761: artifacts/data_ingestion/vpa_data.zip download! with following info: 
Server: GitHub.com
Date: Sat, 04 Jan 2025 05:19:27 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, Accept-Encoding, Accept, X-Requested-With
ETag: W/"93d355fca21bf7ef0f9833b64b41ff1b"
Cache-Control: max-age=0, private, must-revalidate
Strict-Transport-Security: max-age=31536000; includeSubdomains; preload
X-Frame-Options: deny
X-Content-Type-Options: nosniff
X-XSS-Protection: 0
Referrer-Policy: no-ref